# Port Expenses API Example

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
And put your API key in a `SIGNAL_OCEAN_API_KEY` environment variable.

In [ ]:
from signal_ocean import PortExpensesAPI
from datetime import datetime

## Call the port expenses API
The Port Expenses API retrieves port expenses information.

In [ ]:
api = PortExpensesAPI()

#### Get required formula parameters

In [ ]:
port_id = 3153
req_params = api.get_required_formula_parameters(port_id)

#### Get port expenses

In [ ]:
imo = 9867621
port_id = 3153
estimated_time_of_berth = datetime(2020, 2, 27, 17, 48, 11)
port_expenses = api.get_port_expenses(imo, port_id, estimated_time_of_berth=estimated_time_of_berth)

#### Get canal expenses

In [ ]:
canal = 1
imo = 9867621
open_port_id = 3773
load_port_id = 3360
discharge_port_id = 3794
ballast_speed = 12.0
laden_speed = 12.5
operation_status = 0
formula_calculation_date = datetime(2020, 2, 27, 17, 48, 11)
open_date = datetime(1, 1, 1, 0, 0, 0)
load_sail_date = datetime(1, 1, 1, 0, 0, 0)
cargo_type = "Product"
canal_expenses = api.get_canal_expenses(canal, imo, open_port_id, load_port_id, discharge_port_id,
                                        ballast_speed, laden_speed, operation_status,
                                        formula_calculation_date, open_date, load_sail_date, cargo_type)

#### Get port model vessel expenses

In [ ]:
port_id = 3324
vessel_type_id = 5
formula_calculation_date = datetime(2020, 3, 8, 0, 0, 0)
pmv_expenses = api.get_port_model_vessel_expenses(port_id, vessel_type_id, formula_calculation_date)

#### Get canal model vessel expenses

In [ ]:
canal = 1
open_port_id = 3773
load_port_id = 3360
discharge_port_id = 3794
operation_status = 1
formula_calculation_date = datetime(2020, 2, 27, 17, 48, 11)
cmv_expenses = api.get_canal_model_vessel_expenses(canal, open_port_id, load_port_id, discharge_port_id,
                                                    operation_status, formula_calculation_date)